In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, advanced_activations
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.datasets import cifar10
import numpy as np

from keras import backend as K
K.set_image_dim_ordering('th')


import os
import pickle
import numpy as np
from sklearn.model_selection import train_test_split



In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
X=np.concatenate((x_train, x_test), axis=0)
y=np.concatenate((y_train,y_test), axis=0)

In [ ]:
#Splitting the entire dataset randomly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [ ]:
y_test.shape[0]

In [ ]:
#Summary of number of images per class
from collections import Counter
Counter(y_test[:,0])

In [ ]:
#Training network with 12000 images
X_trn, X_tst, y_trn, y_tst = train_test_split(X_test, y_test, test_size=0.3, random_state=5)
#train_data = data[:int((len(X_test)+1)*.80)] #Remaining 80% to training set
#test_data = data[int(len(y_test)*.80+1):] #Splits 20% data to test set

In [ ]:
y_tst.shape[0]

In [ ]:
#Normalizing the values
X_trn = X_trn.astype('float32')
X_tst = X_tst.astype('float32')
X_trn /= 255
X_tst /= 255

In [ ]:
X_trn.shape[0]

In [ ]:
# Convert class vectors to binary class matrices.
y_trn = keras.utils.to_categorical(y_trn, 10)
y_tst = keras.utils.to_categorical(y_tst, 10)

In [ ]:
#Model 3
#Deep CNN  with Maxpooling and leakyrelu

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, advanced_activations
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D

import os
import pickle
import numpy as np

batch_size = 32
num_classes = 10
epochs = 22
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model3.h5'



model_3 = Sequential()

model_3.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_3.add(advanced_activations.LeakyReLU(alpha=0.3))
model_3.add(Conv2D(32, (3, 3)))
model_3.add(advanced_activations.LeakyReLU(alpha=0.3))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.25))

model_3.add(Conv2D(64, (3, 3), padding='same'))
model_3.add(advanced_activations.LeakyReLU(alpha=0.3))
model_3.add(Conv2D(64, (3, 3)))
model_3.add(advanced_activations.LeakyReLU(alpha=0.3))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.25))

model_3.add(Flatten())
model_3.add(Dense(512))
model_3.add(advanced_activations.LeakyReLU(alpha=0.3))
model_3.add(Dropout(0.5))
model_3.add(Dense(num_classes))
model_3.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model_3.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



if not data_augmentation:
    print('Not using data augmentation.')
    model_3.fit(X_trn, y_trn,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_tst, y_tst),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_trn)

    # Fit the model on the batches generated by datagen.flow().
    model_3.fit_generator(datagen.flow(X_trn, y_trn,
                                     batch_size=batch_size),
                        steps_per_epoch=X_trn.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(X_tst, y_tst),
                        workers=4)

# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model_3.evaluate_generator(datagen.flow(X_tst, y_tst,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=X_tst.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model_3.predict_generator(datagen.flow(X_tst, y_tst,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=X_tst.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_tst[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break